# Region Hand-Masking with Firefly

**Contact Author:** Ben Levine
\
**Last verified to run:** 2025-07-18
\
**LSST Science Piplines version:** Weekly 2025_17
\
**Container Size:** small

In this notebook we will use the Firefly image viewer to hand-mask regions of sky, saving these masked regions to a file.

In [ ]:
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay
import numpy as np
import lsst.geom
import astropy.units as u

In [ ]:
afwDisplay.setDefaultBackend('firefly')
afw_display = afwDisplay.Display(frame=1)

In [ ]:
skymap_name =  'lsst_cells_v1'
repo = 'dp1' #'/repo/main'
collection = 'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260'
instrument = "LSSTComCam"

butler = dafButler.Butler(repo, collections=collection, skymap=skymap_name, 
                                     instrument=instrument)

## There are two ways to select the viewing area:

1. **Select from Coordinates**: insert the target RA and Dec into the *RA_DEG* and *DEC_DEG* fields below. The cell will automatically identify the image tract and patch to query.
2. **Select from Tract/Patch**: insert the tract and patch id numbers into the *TRACT* and *PATCH* fields below.

In [ ]:
SELECT_COORDINATES = False # Should we query based on the given coordinates? (Or based on a given tract and patch?)

RA_DEG, DEC_DEG = 37.86501659859067, 6.982204815599694
TRACT, PATCH = 10463, 81
BAND = 'r' # <-- You can change the band here if you'd like.

# -----------------------

if SELECT_COORDINATES:
    my_spherePoint = lsst.geom.SpherePoint(RA_DEG*lsst.geom.degrees,
                                           DEC_DEG*lsst.geom.degrees)
    skymap = butler.get('skyMap')
    tract = skymap.findTract(my_spherePoint)
    patch = tract.findPatch(my_spherePoint)
    TRACT = tract.tract_id
    PATCH = patch.getSequentialIndex()
    
print('tract id: ', TRACT)
print('patch id: ', PATCH)
coadd = butler.get("deep_coadd", tract=TRACT, patch=PATCH, band=BAND) 

afw_display.mtv(coadd)

## Instructions for Masking:

At this point, you should have a second tab opened with the Firefly window showing. Adding a region marker is simple:

1. Click the tool icon on the toolbar (upper right, has a picture of a hammer and wrench).
2. In the dropdown menu, under the "Layers" row, click the rightmost icon ("Overlay Markers and Instrument Footprints"; shaped like a circle).
3. In the new dropdown menu, click "Add Marker."

The marker can be moved an resized just as in DS9. You can add as many markers to the image as you like. 

Once finished, the region file can be saved as follows:

1. Click the tool icon on the toolbar (upper right, has a picture of a hammer and wrench).
2. In the dropdown menu, under the "Save/Restore/Info" row, click the leftmost icon (save).
3. Select "region file" for the file type. Then click save.
4. Send the completed region file to Ben Levine!

**Extra tip:** The image overlays can be very distracting. You can turn them off in the "Layers" tab in the toolbar (upper right, has a picture of two squares kind of stacked on top of one another). 

### Catalog overlay

In [ ]:
# Might need to update some of these flag names

obj_cols = [f'{BAND}_centroid_x', f'{BAND}_centroid_y', f'{BAND}_cModelFlux', f'{BAND}_gaap1p0Flux',
            'detect_isPrimary', 'detect_isTractInner', 'merge_peak_sky', 'detect_isDeblendedSource', #this row might need to be removed
            f'{BAND}_centroid_flag', f'{BAND}_cModel_flag', f'{BAND}_gaapFlux_flag',
            f'{BAND}_extendedness', 'refExtendedness']

In [ ]:
obj = butler.get("object_patch", tract=TRACT, patch=PATCH, band=BAND,
                     skymap='lsst_cells_v1', parameters={"columns":obj_cols})

In [ ]:
band_mag = u.nJy.to(u.ABmag, obj[f'{BAND}_gaap1p0Flux'])
# band_mag = u.nJy.to(u.ABmag, obj[f'{BAND}_cModelFlux'])

In [ ]:
# Combine multiple filters with .reduce
good_obj = np.logical_and.reduce((~obj['merge_peak_sky'], ~obj[f'{BAND}_gaapFlux_flag'],
                                  obj['detect_isDeblendedSource'], band_mag<=27, obj['refExtendedness']==1))
primary_obj = obj[good_obj]

In [ ]:
afw_display.erase()

In [ ]:
rx = primary_obj['r_centroid_x'].astype(int)
ry = primary_obj['r_centroid_y'].astype(int)

with afw_display.Buffering():
    for i in range(len(rx)):
        afw_display.dot('o', rx[i], ry[i], size=5, ctype='red')